In [1]:
# !pip install selenium
import logging
import re
import json
import time
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
def setup_logger(name='DocDocParser'): # логгирование
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(options=chrome_options)
    return driver

def is_valid_doctor_link(href): # проверка ссылок на корректность
    if not href:
        return False
    if len(href.split('#')) > 1:
        return False
    match = re.search(r'/doctor/([A-ZА-Я][^/?]*)', href)
    return match is not None

In [3]:
def parse_name(soup):
    try:
        name_elem = soup.find('div', {'data-testid': 'doctor__fio'})
        return name_elem.text.strip() if name_elem else None
    except:
        return None

def parse_speciality(soup):
    try:
        speciality_container = soup.find('div', {'aria-label': 'Выбор специальности врача'})
        if speciality_container:
            speciality_spans = speciality_container.find_all('span', {'class': 'sdsClinicChip__t138vcdl sdsClinicChip__hhhycyd'})
            if speciality_spans:
                return ', '.join([x.text.strip() for x in speciality_spans])

        speciality_elem = soup.find('li', {'data-testid': 'summary__speciality'})
        if speciality_elem:
            speciality_span = speciality_elem.find('span', {'class': 'tzjpv3j'})
            return speciality_span.text.strip() if speciality_span else None
    except:
        pass
    return None

def parse_clinics(soup):
    clinics = []

    try:
        # Первый вариант - контейнер с выбором клиники
        clinics_container = soup.find('div', {'aria-label': 'Выбор клиники'})
        if clinics_container:
            all_clinics = clinics_container.find_all('span', {'class': 'sdsClinicChip__c1yy1ila'})
            for x in all_clinics:
                name_elem = x.find('span', {'class': 'sdsClinicChip__t138vcdl sdsClinicChip__hhhycyd'})
                address_elem = x.find('span', {'class': 'sdsClinicChip__s2rlal3 sdsClinicChip__hhhycyd'})
                metro_elem = x.find('span', {'class': 'sdsSubway_13f0c476'})

                clinics.append({
                    'name': name_elem.text.strip() if name_elem else None,
                    'address': address_elem.text.strip() if address_elem else None,
                    'metro': metro_elem.text.strip() if metro_elem else None
                })

            if clinics:
                return clinics

        # Второй вариант - одиночная клиника
        clinic = soup.find('div', {'data-testid': 'doctor-page__clinic'})
        if clinic:
            name_elem = clinic.find('a', {'data-testid': 'doctor-page__clinic-name'})
            address_elem = clinic.find('div', {'data-testid': 'doctor-page__address-name'})
            metro_elem = clinic.find('span', {'class': 'sdsSubway_a29aae9d'})

            return [{
                'name': name_elem.text.strip() if name_elem else None,
                'address': address_elem.text.strip() if address_elem else None,
                'metro': metro_elem.text.strip() if metro_elem else None
            }]
    except:
        pass

    return []

def parse_price(soup):
    try:
        price_elems = [x.text.strip() for x in soup.find_all('div', {'data-testid': 'doctor-page__price-full'}) if '₽' in x.text]
        if price_elems:
            return price_elems[-1]

        price_elems = [x.text.strip() for x in soup.find_all('div', {'data-testid': 'doctor-page__price-final'}) if '₽' in x.text]
        if price_elems:
            return price_elems[-1]
    except:
        pass
    return None

def parse_experience(soup):
    try:
        experience_container = soup.find('div', {'data-testid': 'doctor__nameplate-experience'})
        if experience_container:
            experience_elem = experience_container.find('div', {'class': 't7amcxk'})
            return experience_elem.text.strip() if experience_elem else None
    except:
        pass
    return None

def parse_rating(soup):
    try:
        rating_container = soup.find('div', {'data-testid': 'doctor__rating-stars-mobile'})
        if rating_container:
            rating_elem = rating_container.find('span', {'class': 'sdsRatingStarsValue_695f7498'})
            return rating_elem.text.strip() if rating_elem else None
    except:
        pass
    return None

def parse_review_count(soup):
    try:
        review_container = soup.find('div', {'data-testid': 'doctor__nameplate-reviews-count'})
        if review_container:
            review_elem = review_container.find('div', {'class': 't7amcxk'})
            if review_elem:
                try:
                    return int(re.search(r'\d+', review_elem.text.strip()).group())
                except:
                    pass
    except:
        pass
    return 0

def parse_reviews(soup, review_count):
    if review_count <= 0:
        return None

    try:
        script_elem = soup.find('script', {'id': '__NEXT_DATA__'})
        if script_elem:
            data = json.loads(script_elem.text)
            reviews_data = data.get('props', {}).get('pageProps', {}).get('preloadedState', {}).get('doctorPage', {}).get('doctor', {}).get('reviewsForSeo', [])

            reviews = []
            for review in reviews_data:
                reviews.append({
                    'rate': review.get('rating', {}).get('label', None),
                    'date': review.get('date', None),
                    'comment': review.get('text', None),
                    'clinic': review.get('clinic', {}).get('name', None)
                })

            return reviews if reviews else None
    except:
        pass

    return None

def parse_patient_ages(soup):
    kids = False
    adults = False

    try:
        patients_container = soup.find('div', {'aria-label': 'Выбор возраста пациента'})
        if patients_container:
            patient_spans = patients_container.find_all('span', {'class': 'sdsClinicChip__t138vcdl sdsClinicChip__hhhycyd'})
            patients_text = ''.join([x.text.lower() for x in patient_spans])
            kids = 'дети' in patients_text
            adults = 'взрослые' in patients_text
            return kids, adults

        age_elem = soup.find('li', {'data-testid': 'summary__age'})
        if age_elem:
            age_span = age_elem.find('span', {'class': 'tzjpv3j'})
            if age_span:
                patient_text = age_span.text.strip().lower()
                kids = 'дети' in patient_text
                adults = 'взрослые' in patient_text
    except:
        pass

    return kids, adults

In [4]:
def parse_doctor(src, doctor_url):
    soup = BeautifulSoup(src, "lxml")

    name = parse_name(soup)
    speciality = parse_speciality(soup)
    clinics = parse_clinics(soup)
    price = parse_price(soup)
    experience = parse_experience(soup)
    rating = parse_rating(soup)
    review_count = parse_review_count(soup)
    reviews = parse_reviews(soup, review_count)
    kids, adults = parse_patient_ages(soup)

    return {
        'name': name,
        'link': doctor_url,
        'speciality': speciality,
        'clinics': clinics if clinics else None,
        'price': price,
        'experience': experience,
        'rating': rating,
        'review_count': review_count,
        'reviews': reviews,
        'is_kids': kids,
        'is_adults': adults
    }

In [5]:
def get_doctor_links_from_page(driver, logger):
    all_links = driver.find_elements(By.TAG_NAME, "a")
    seen_links = set()
    doctor_links = []

    for link in all_links:
        try:
            href = link.get_attribute('href')
            if href and '/doctor/' in href:
                normalized_href = href.split('#')[0].split('?')[0]
                if normalized_href not in seen_links and is_valid_doctor_link(href):
                    seen_links.add(normalized_href)
                    doctor_links.append(href)
        except:
            pass

    logger.info(f"Найдено валидных ссылок: {len(doctor_links)}")
    return doctor_links

def get_next_page_link(driver):
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="pagination-next"]')
        return next_button.get_attribute('href')
    except:
        return None

def parse_doctors_on_page(driver, doctor_links, max_doctors_per_page, logger):
    doctors_data = []
    success_count = 0

    doctors_to_parse = doctor_links[:max_doctors_per_page] if max_doctors_per_page else doctor_links

    for i, doctor_url in enumerate(doctors_to_parse):
        try:
            driver.get(doctor_url)
            time.sleep(1)

            doctor_data = parse_doctor(driver.page_source, doctor_url)
            doctors_data.append(doctor_data)
            
            with open('doctor_data.jsonl', 'a', encoding='utf-8') as f:
                f.write(json.dumps(doctor_data, ensure_ascii=False) + '\n')
                
            success_count += 1

            driver.back()
            time.sleep(1)

        except Exception as e:
            logger.error(f"Ошибка при парсинге врача {doctor_url}: {e}")
            continue

    logger.info(f"Успешно спарсено врачей: {success_count}/{len(doctors_to_parse)}")
    return doctors_data

In [6]:
def prepare_csv_data(doctors_data):
    csv_data = []

    for doctor in doctors_data:
        row = {
            'name': doctor.get('name', None),
            'speciality': doctor.get('speciality', None),
            'experience': doctor.get('experience', None),
            'rating': doctor.get('rating', None),
            'review_count': doctor.get('review_count', 0),
            'price': doctor.get('price', None),
            'is_kids': doctor.get('is_kids', False),
            'is_adults': doctor.get('is_adults', False),
            'link': doctor.get('link', None),
            'clinics_count': len(doctor.get('clinics', [])) if doctor.get('clinics', []) else 0,
        }

        clinics = doctor.get('clinics', [])
        for i, clinic in enumerate(clinics[:3]):
            row[f'clinic_{i+1}_name'] = clinic.get('name', None)
            row[f'clinic_{i+1}_address'] = clinic.get('address', None)
            row[f'clinic_{i+1}_metro'] = clinic.get('metro', None)

        for i in range(len(clinics), 3):
            row[f'clinic_{i+1}_name'] = None
            row[f'clinic_{i+1}_address'] = None
            row[f'clinic_{i+1}_metro'] = None

        csv_data.append(row)

    return csv_data

def save_to_dataset(doctors_data, filename_prefix, logger):
    if not doctors_data:
        if logger:
            logger.info("Ошибка: нет данных для сохранения")
        return None

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    json_filename = f"{filename_prefix}_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(doctors_data, f, ensure_ascii=False, indent=2)

    if logger:
        logger.info(f"Полные данные сохранены в {json_filename}")

    csv_data = prepare_csv_data(doctors_data)
    csv_filename = f"{filename_prefix}_{timestamp}.csv"
    df = pd.DataFrame(csv_data)
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

    if logger:
        logger.info(f"Датасет сохранен в {csv_filename}")

    return df

In [7]:
class SeleniumDocDocParser:
    def __init__(self, start = 0):
        self.base_url = "https://docdoc.ru"
        self.doctors_url = "https://docdoc.ru/doctor/endokrinolog" # Изначальная ссылка
        self.doctors_data = []
        self.logger = setup_logger('DocDocParser')
        self.start = start

    def setup_driver(self):
        return setup_driver()

    def parse_doctor(self, src, doctor_url):
        return parse_doctor(src, doctor_url)

    def is_valid_doctor_link(self, href):
        return is_valid_doctor_link(href)

    def get_doctor_links_from_page(self, driver):
        return get_doctor_links_from_page(driver, self.logger)

    def get_next_page_link(self, driver):
        return get_next_page_link(driver)

    def save_to_dataset(self, filename_prefix='doctors'):
        return save_to_dataset(self.doctors_data, filename_prefix, self.logger)

    def parse_multiple_pages(self, max_pages=1, max_doctors_per_page=None):
        driver = self.setup_driver()
        page_count = 0

        try:
            self.logger.info("Parsing started.")

            while page_count < max_pages:
                page_count += 1
                current_url = f"{self.doctors_url}/page/{page_count + self.start}"
                self.logger.info(f"Page {page_count + self.start} loading...")
                self.logger.info(f"URL: {current_url}")

                driver.get(current_url)
                time.sleep(1)

                self.logger.info("Searching the links")
                doctor_links = self.get_doctor_links_from_page(driver)

                if not doctor_links:
                    self.logger.info(f"Не найдено валидных ссылок на врачей. Переход к следующей странице.")
                    continue

                # Парсим врачей с текущей страницы
                self.logger.info(f"Парсинг врачей на странице {page_count}...")
                page_doctors_data = parse_doctors_on_page(
                    driver, doctor_links, max_doctors_per_page, self.logger
                )
                self.doctors_data.extend(page_doctors_data)

            # Сохраняем все собранные данные
            self.logger.info(f"Сохраненяем все собранные данные ({len(self.doctors_data)} записей)...")
            if self.doctors_data:
                self.save_to_dataset('doctors'+str(self.start))
                self.logger.info("Парсинг завершён!")
                self.logger.info(f"Всего считано врачей: {len(self.doctors_data)}")
            else:
                self.logger.info("Ошибка парсинга: врачи отсутствуют")

        except Exception as e:
            self.logger.error(f"Ошибка во время парсинга: {e}")
        finally:
            driver.quit()
            self.logger.info("Браузер закрыт")

In [8]:
parser = SeleniumDocDocParser(152) # В скобки номер страницы
parser.parse_multiple_pages(max_pages=155, max_doctors_per_page=20) # В первый параметр количество страниц

2025-11-04 03:52:47,923 - INFO -  Parsing started.
2025-11-04 03:52:47,924 - INFO - 
Page 153 loading...
2025-11-04 03:52:47,924 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/153
2025-11-04 03:52:50,671 - INFO - 
Searching the links
2025-11-04 03:52:51,182 - INFO - Найдено валидных ссылок: 20
2025-11-04 03:52:51,182 - INFO -  Парсинг врачей (максимум 20) на странице 1...
2025-11-04 03:54:27,476 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 03:54:27,476 - INFO - 
Page 154 loading...
2025-11-04 03:54:27,477 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/154
2025-11-04 03:54:29,810 - INFO - 
Searching the links
2025-11-04 03:54:30,878 - INFO - Найдено валидных ссылок: 20
2025-11-04 03:54:30,879 - INFO -  Парсинг врачей (максимум 20) на странице 2...
2025-11-04 03:56:11,656 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 03:56:11,659 - INFO - 
Page 155 loading...
2025-11-04 03:56:11,661 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/155
2025-11-04 03:56:

2025-11-04 04:27:01,992 - INFO - Найдено валидных ссылок: 20
2025-11-04 04:27:01,992 - INFO -  Парсинг врачей (максимум 20) на странице 21...
2025-11-04 04:28:41,579 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 04:28:41,581 - INFO - 
Page 174 loading...
2025-11-04 04:28:41,582 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/174
2025-11-04 04:28:44,130 - INFO - 
Searching the links
2025-11-04 04:28:44,954 - INFO - Найдено валидных ссылок: 20
2025-11-04 04:28:44,955 - INFO -  Парсинг врачей (максимум 20) на странице 22...
2025-11-04 04:30:22,287 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 04:30:22,288 - INFO - 
Page 175 loading...
2025-11-04 04:30:22,289 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/175
2025-11-04 04:30:25,374 - INFO - 
Searching the links
2025-11-04 04:30:26,206 - INFO - Найдено валидных ссылок: 20
2025-11-04 04:30:26,207 - INFO -  Парсинг врачей (максимум 20) на странице 23...
2025-11-04 04:32:10,526 - INFO - Успешно спарсено врачей: 20/2

2025-11-04 05:03:25,246 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/194
2025-11-04 05:03:27,930 - INFO - 
Searching the links
2025-11-04 05:03:29,261 - INFO - Найдено валидных ссылок: 20
2025-11-04 05:03:29,262 - INFO -  Парсинг врачей (максимум 20) на странице 42...
2025-11-04 05:05:14,447 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 05:05:14,449 - INFO - 
Page 195 loading...
2025-11-04 05:05:14,451 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/195
2025-11-04 05:05:16,814 - INFO - 
Searching the links
2025-11-04 05:05:17,676 - INFO - Найдено валидных ссылок: 20
2025-11-04 05:05:17,677 - INFO -  Парсинг врачей (максимум 20) на странице 43...
2025-11-04 05:07:01,494 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 05:07:01,496 - INFO - 
Page 196 loading...
2025-11-04 05:07:01,497 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/196
2025-11-04 05:07:04,062 - INFO - 
Searching the links
2025-11-04 05:07:04,883 - INFO - Найдено валидных ссылок: 20
2025-

2025-11-04 05:40:36,524 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 05:40:36,526 - INFO - 
Page 215 loading...
2025-11-04 05:40:36,528 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/215
2025-11-04 05:40:39,043 - INFO - 
Searching the links
2025-11-04 05:40:39,861 - INFO - Найдено валидных ссылок: 20
2025-11-04 05:40:39,862 - INFO -  Парсинг врачей (максимум 20) на странице 63...
2025-11-04 05:42:22,667 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 05:42:22,669 - INFO - 
Page 216 loading...
2025-11-04 05:42:22,671 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/216
2025-11-04 05:42:25,245 - INFO - 
Searching the links
2025-11-04 05:42:26,354 - INFO - Найдено валидных ссылок: 20
2025-11-04 05:42:26,355 - INFO -  Парсинг врачей (максимум 20) на странице 64...
2025-11-04 05:44:12,795 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 05:44:12,796 - INFO - 
Page 217 loading...
2025-11-04 05:44:12,797 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/217
20

2025-11-04 06:14:51,901 - INFO - Найдено валидных ссылок: 20
2025-11-04 06:14:51,902 - INFO -  Парсинг врачей (максимум 20) на странице 83...
2025-11-04 06:16:33,825 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 06:16:33,827 - INFO - 
Page 236 loading...
2025-11-04 06:16:33,829 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/236
2025-11-04 06:16:36,786 - INFO - 
Searching the links
2025-11-04 06:16:37,619 - INFO - Найдено валидных ссылок: 20
2025-11-04 06:16:37,620 - INFO -  Парсинг врачей (максимум 20) на странице 84...
2025-11-04 06:18:20,326 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 06:18:20,327 - INFO - 
Page 237 loading...
2025-11-04 06:18:20,329 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/237
2025-11-04 06:18:22,938 - INFO - 
Searching the links
2025-11-04 06:18:23,690 - INFO - Найдено валидных ссылок: 20
2025-11-04 06:18:23,691 - INFO -  Парсинг врачей (максимум 20) на странице 85...
2025-11-04 06:20:03,611 - INFO - Успешно спарсено врачей: 20/2

2025-11-04 06:51:32,494 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/256
2025-11-04 06:51:35,188 - INFO - 
Searching the links
2025-11-04 06:51:36,705 - INFO - Найдено валидных ссылок: 20
2025-11-04 06:51:36,706 - INFO -  Парсинг врачей (максимум 20) на странице 104...
2025-11-04 06:53:13,983 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 06:53:13,984 - INFO - 
Page 257 loading...
2025-11-04 06:53:13,985 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/257
2025-11-04 06:53:18,144 - INFO - 
Searching the links
2025-11-04 06:53:18,873 - INFO - Найдено валидных ссылок: 20
2025-11-04 06:53:18,874 - INFO -  Парсинг врачей (максимум 20) на странице 105...
2025-11-04 06:54:53,993 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 06:54:53,995 - INFO - 
Page 258 loading...
2025-11-04 06:54:53,997 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/258
2025-11-04 06:54:56,613 - INFO - 
Searching the links
2025-11-04 06:54:57,465 - INFO - Найдено валидных ссылок: 20
202

2025-11-04 07:27:14,520 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 07:27:14,521 - INFO - 
Page 277 loading...
2025-11-04 07:27:14,522 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/277
2025-11-04 07:27:16,933 - INFO - 
Searching the links
2025-11-04 07:27:18,164 - INFO - Найдено валидных ссылок: 20
2025-11-04 07:27:18,165 - INFO -  Парсинг врачей (максимум 20) на странице 125...
2025-11-04 07:28:53,095 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 07:28:53,097 - INFO - 
Page 278 loading...
2025-11-04 07:28:53,098 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/278
2025-11-04 07:28:55,710 - INFO - 
Searching the links
2025-11-04 07:28:57,659 - INFO - Найдено валидных ссылок: 20
2025-11-04 07:28:57,660 - INFO -  Парсинг врачей (максимум 20) на странице 126...
2025-11-04 07:30:32,239 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 07:30:32,241 - INFO - 
Page 279 loading...
2025-11-04 07:30:32,243 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/279


2025-11-04 08:00:57,380 - INFO - Найдено валидных ссылок: 20
2025-11-04 08:00:57,381 - INFO -  Парсинг врачей (максимум 20) на странице 145...
2025-11-04 08:02:39,338 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 08:02:39,339 - INFO - 
Page 298 loading...
2025-11-04 08:02:39,341 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/298
2025-11-04 08:02:42,070 - INFO - 
Searching the links
2025-11-04 08:02:43,775 - INFO - Найдено валидных ссылок: 20
2025-11-04 08:02:43,775 - INFO -  Парсинг врачей (максимум 20) на странице 146...
2025-11-04 08:04:23,717 - INFO - Успешно спарсено врачей: 20/20
2025-11-04 08:04:23,719 - INFO - 
Page 299 loading...
2025-11-04 08:04:23,720 - INFO -    URL: https://docdoc.ru/doctor/terapevt/page/299
2025-11-04 08:04:26,370 - INFO - 
Searching the links
2025-11-04 08:04:28,038 - INFO - Найдено валидных ссылок: 20
2025-11-04 08:04:28,039 - INFO -  Парсинг врачей (максимум 20) на странице 147...
2025-11-04 08:06:05,292 - INFO - Успешно спарсено врачей: 2